# Minimal cadCAD

Danilo Lessa Bernadineli (danilo@block.science)

Emanuel Lima (emanuel@block.science)



This notebook is a minimal implementation of the basic cadCAD structure

---

Github version: https://github.com/cadCAD-org/snippets


Colab version: https://colab.research.google.com/drive/11MEtzHlWLfiflnKg4FaI4cBkvL_6wUtD#scrollTo=9weVx_a50reF


In [ ]:
from random import random
from tqdm.auto import tqdm

# Mutable state
variables = {
    'prey_population': 0.5,
    'predator_population': 0.5
}

# Having variables as keys assures
# consistency.
# If it were lists, then a user could provide
# several functions for the same variable,
# which would make the update concurrent.
# wrong_substep_block = [
#     'prey_population': None,
#     'prey_population': None
# ]

### Parameters ###

# Prey Growth Rate
ALPHA = 2 / 3

# Prey Death Rate
BETA = 4 / 3

# Predator Growth Rate
DELTA = 1.0

# Predator Death Rate
GAMMA = 1.0

# Time interval
dt = 0.1

### State Update Functions ###

def prey_change(state: dict) -> float:
  x = state['prey_population']
  y = state['predator_population']
  dx = ALPHA * x - BETA * x * y
  value = max(x + dx * dt, 0)
  return value

def predator_change(state: dict) -> float:
  x = state['prey_population']
  y = state['predator_population']
  dy = DELTA * x * y - GAMMA * y
  value = max(y + dy * dt, 0)
  return value

def stochastic_process(state: dict) -> float:
  x = state['prey_population']
  value = x * (1 + random() / 1000)
  return value

### Partial State Update Block ###

## SUBs ##
# Represents intermediate state changes between t and t+1

substep_block_1 = {
    'prey_population': prey_change,
    'predator_population': predator_change,
}

substep_block_2 = {
    'prey_population': stochastic_process
}

# Represents a state change from t -> t+1
timestep_block = [
                  substep_block_1,
                  substep_block_2
]

# Simulation config

timestep_count = int(1e3)

### Simulation Execution ###

current_state = variables.copy()
results = []

# Iteration count: (T, S, f given S)
for timestep in tqdm(range(timestep_count)):

  # Iteration count: (S, f given S)
  for substep, substep_block  in enumerate(timestep_block):

    # We copy for making sure that the old state
    # which is history tracked is not changed
    new_state = current_state.copy()

    # Iteration count: f given S
    for variable, variable_update_function in substep_block.items():
      new_state[variable] = variable_update_function(current_state)

    # Append timestep and substep
    new_state.update(timestep=timestep, substep=substep)
    
    results.append(new_state)
    current_state = new_state


### Prepare results ###

import pandas as pd
df = pd.DataFrame(results)
df

,prey_population,predator_population,timestep,substep
0,0.500000,4.750000e-01,0,0
1,0.500476,4.750000e-01,0,1
2,0.502144,4.512726e-01,1,0
3,0.502523,4.512726e-01,1,1
4,0.505788,4.288228e-01,2,0
...,...,...,...,...
1995,0.000000,4.158761e-08,997,1
1996,0.000000,3.742885e-08,998,0
1997,0.000000,3.742885e-08,998,1
1998,0.000000,3.368596e-08,999,0


In [ ]:
import plotly.express as px

px.scatter(df,
           x='predator_population',
           y='prey_population',
           color='timestep')